In [1]:
import tensorflow as tf
from tensorflow import set_random_seed
import numpy as np
import keras
from keras.layers import LSTM, Masking, Input, Dense
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [49]:
tf.reset_default_graph()

# Create input data
X = np.random.randn(2, 10, 8)

# The second example is of length 6 
X[1,6:] = 0
X_lengths = [10, 6]
X = tf.constant(X)

cell = tf.nn.rnn_cell.LSTMCell(num_units=2, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=2,
    feed_dict=None)

assert result[0]["outputs"].shape == (2, 10, 2)
print(result[0]["outputs"])

# Outputs for the second example past past length 6 should be 0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()
print(result[0]["last_states"])

[[[ 0.0553893  -0.05387677]
  [ 0.07814491  0.01852085]
  [ 0.45439345  0.07240493]
  [ 0.05437639  0.19665424]
  [ 0.10917992  0.10262374]
  [-0.04669672 -0.05310226]
  [-0.255147    0.13007151]
  [-0.1825807   0.25034467]
  [ 0.14650409  0.02037374]
  [-0.10142151 -0.01488104]]

 [[-0.01223021 -0.10013643]
  [ 0.00505111 -0.01831526]
  [-0.02811303  0.05176523]
  [ 0.20172692  0.12557475]
  [ 0.38125063 -0.02640679]
  [ 0.09388913 -0.14412151]
  [ 0.          0.        ]
  [ 0.          0.        ]
  [ 0.          0.        ]
  [ 0.          0.        ]]]
LSTMStateTuple(c=array([[-0.13138615, -0.04730978],
       [ 0.49442086, -0.29152655]]), h=array([[-0.10142151, -0.01488104],
       [ 0.09388913, -0.14412151]]))


In [10]:
def model_predict(model, x):
    print("Model predict")
    output = model.predict(x)[0]
    print(f'Shape of outputs: {output.shape}')
    print(output)
    print(model.predict(x)[1])
    print(model.predict(x)[2])


print("Keras with Masking")

np.random.seed(42)
set_random_seed(42)
# Create input data
x = np.random.randn(2, 10, 8)

# The second example is of length 6 
x[1,6:] = 0
x_lengths = [10, 6]

inputs = Input(shape=[10, 8])
X = Masking(mask_value=0)(inputs)
X, state_h, state_c = LSTM(3, return_sequences=True,return_state=True)(X)
model = Model(inputs, [X, state_h, state_c])
output = model.predict(x)[0]
model_predict(model, x)

#X, state_h, state_c = LSTM(3, return_sequences=True,return_state=True)(X)
#model = Model(inputs, [X, state_h, state_c])
#model_predict(model, x)

X = Dense(1)(X)
model_final = Model(inputs, X)
print("Model predict")
output = model_final.predict(x)
print(f'Shape of outputs: {output.shape}')
print(output)


model_final.summary()
for layer in model_final.layers:
    for w in layer.get_weights():
        print(w.shape)
        print(w)
        



Keras with Masking
Model predict
Shape of outputs: (2, 10, 3)
[[[-0.3169843  -0.05809069 -0.1289793 ]
  [ 0.03166906  0.13537185  0.02060456]
  [ 0.22555707  0.1820613   0.09070159]
  [ 0.15419526  0.12339647  0.31557813]
  [ 0.28644127  0.1210888   0.1524397 ]
  [ 0.12700213  0.073625    0.12862103]
  [-0.14187782 -0.03728769 -0.06768028]
  [-0.02682975 -0.04723642 -0.12080718]
  [-0.10736605  0.01618     0.11987579]
  [-0.12158591  0.23908715  0.3620818 ]]

 [[ 0.14483084 -0.2736832  -0.13532183]
  [ 0.15770528 -0.13724373 -0.21121924]
  [ 0.1383527  -0.02530398 -0.23079664]
  [ 0.2291775  -0.14347799 -0.12989558]
  [ 0.4934633  -0.06992987 -0.01199105]
  [ 0.11602164 -0.12029754 -0.00394765]
  [ 0.11602164 -0.12029754 -0.00394765]
  [ 0.11602164 -0.12029754 -0.00394765]
  [ 0.11602164 -0.12029754 -0.00394765]
  [ 0.11602164 -0.12029754 -0.00394765]]]
[[-0.12158591  0.23908715  0.3620818 ]
 [ 0.11602164 -0.12029754 -0.00394765]]
[[-0.43912894  0.672187    0.6925829 ]
 [ 0.43454343 -0

In [41]:
model_final.summary()
model_final.compile(loss='mse', optimizer='Adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, 10, 8)             0         
_________________________________________________________________
masking_18 (Masking)         (None, 10, 8)             0         
_________________________________________________________________
lstm_34 (LSTM)               [(None, 10, 2), (None, 2) 88        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 3         
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [24]:
print("Keras without Masking")

np.random.seed(42)
set_random_seed(42)
# Create input data
x = np.random.randn(2, 10, 8)

# The second example is of length 6 
x[1,6:] = 0
x_lengths = [10, 6]

inputs = Input(shape=[10, 8])
# X = Masking(mask_value=0)(inputs)
X, state_h, state_c = LSTM(2, return_sequences=True,return_state=True)(inputs)
model = Model(inputs, [X, state_h, state_c])

output = model.predict(x)[0]
print(f'Shape of outputs: {output.shape}')
print(output)

Keras without Masking
Shape of outputs: (2, 10, 2)
[[[ 0.3125334   0.02155844]
  [ 0.00243016  0.0759569 ]
  [-0.03230448 -0.2080449 ]
  [-0.22587523 -0.01980554]
  [-0.07068695 -0.0240958 ]
  [-0.26760152  0.11328933]
  [-0.25674865 -0.01973266]
  [-0.15950948  0.03653106]
  [-0.03294701  0.07563439]
  [-0.00444926  0.02212346]]

 [[ 0.24402463  0.15404473]
  [ 0.13173574  0.15281159]
  [ 0.01409378  0.24500534]
  [-0.09657343  0.22761787]
  [ 0.08079094  0.21155256]
  [ 0.0600839   0.20679118]
  [ 0.05763446  0.11870763]
  [ 0.0499833   0.09624448]
  [ 0.04293518  0.07692941]
  [ 0.03640639  0.06170126]]]
